In [2]:
%reload_ext autoreload
%autoreload 2

In [3]:
# regular imports
import sys
sys.path.append('..')
import matplotlib.pyplot as plt
%matplotlib inline

import albumentations as A
from albumentations.pytorch import ToTensorV2

# Lightning import
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import Callback, LearningRateMonitor, ModelCheckpoint
from pytorch_lightning.loggers import WandbLogger
# from pl_bolts.datamodules.mnist_datamodule import MNISTDataModule

# PyTorch imports
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader  # , Dataset
print(f'Cuda available: {torch.cuda.is_available()}')

import wandb
wandb.login()

# internal imports
from src.callbacks import ImagePredictionLogger
from src.dataset import alb_to_torch_aug, MNISTDataModule  # , ComposeMany
from src.losses import log_softmax
from src.models import accuracy, CNN, BaseLitModel, SimSiam
from src.utils import sweep_iteration

Cuda available: True


wandb: Currently logged in as: alkalait (use `wandb login --relogin` to force relogin)


---

In [4]:
# Augmentator

AUG_KWARGS = dict(border_mode=A.cv2.BORDER_CONSTANT, value=0,
                  interpolation=A.cv2.INTER_LANCZOS4)

train_transforms = A.Compose([
# train_transforms = ComposeMany([
    #A.RandomCrop(width=24, height=24),
    #A.HorizontalFlip(p=0.5),
    #A.GridDistortion(p=0.5, distort_limit=.3, **AUG_KWARGS),
    A.ElasticTransform(p=0.5, sigma=1, alpha=3, alpha_affine=0, **AUG_KWARGS),
    A.ElasticTransform(p=0.5, sigma=1, alpha=1, alpha_affine=3, **AUG_KWARGS),
    A.ShiftScaleRotate(p=1.0, scale_limit=.2, rotate_limit=0, **AUG_KWARGS),
    A.ShiftScaleRotate(p=1.0, scale_limit=0, rotate_limit=25, **AUG_KWARGS),
    #A.CoarseDropout(p=1.0, max_holes=8, max_height=4, max_width=4,
    #                min_holes=1, min_height=4, min_width=4),
    #A.RandomBrightnessContrast(p=0.2),
    #A.Blur(blur_limit=4),
    A.Normalize(mean=(0.0,), std=(1,)),  # , max_pixel_value=255),
    ToTensorV2()
])#, n_aug=2)

In [ ]:
proj = 'SimSiam-Lightning'

## Lightning datamodule. Comes with its own train / val / test dataloader.
mnist = MNISTDataModule('../data/', batch_size=512, train_transforms=train_transforms)
mnist.prepare_data()
mnist.setup()

## Backbone arch
cnn = CNN(C=mnist.dims[0], num_classes=mnist.num_classes)

## Metrics: (name of log-entry, metric)
metrics = (('acc', accuracy),)

model = BaseLitModel(
    datamodule=mnist, backbone=cnn, loss_func=log_softmax, metrics=metrics,
    lr=1e-3, flood_height=0.03
)

In [ ]:
wandb_logger = WandbLogger(project=proj, job_type='train')
callbacks = [
    LearningRateMonitor(),  # log the LR
    ImagePredictionLogger(mnist.val_dataloader(batch_size=32), n_samples=32),
]

trainer = Trainer(
    max_epochs=200, gpus=-1,  # all GPUs
    logger=wandb_logger, callbacks=callbacks,
    accumulate_grad_batches=1, gradient_clip_val=0,  # 0.5
    progress_bar_refresh_rate=20,
    #fast_dev_run=True,
)

In [ ]:
trainer.fit(model, mnist)

---

In [ ]:
trainer.test()

wandb.finish()

---

# LR finder

In [ ]:
# # Learning rate finder
# lr_finder = trainer.tuner.lr_find(model, num_training=3000, mode='linear', max_lr=1e-2)
# # lr_finder.results  # Results can be found in
# fig = lr_finder.plot(suggest=True)
# lr_finder.suggestion()
# model.hparams.lr = new_lr  # update hparams of the model

# Hyperparameter sweep 

In [ ]:
# from src.sweeps import sweep_config

# sweep_id = wandb.sweep(sweep_config, project=proj)

# wandb.agent(sweep_id, function=sweep_iteration, project=proj)

---

# SimSiam

In [5]:
# Augmentator

AUG_KWARGS = dict(border_mode=A.cv2.BORDER_CONSTANT, value=0,
                  interpolation=A.cv2.INTER_LANCZOS4)

train_transforms = A.Compose([
# train_transforms = ComposeMany([
    A.ElasticTransform(p=0.5, sigma=1, alpha=3, alpha_affine=0, **AUG_KWARGS),
    A.ElasticTransform(p=0.5, sigma=1, alpha=1, alpha_affine=3, **AUG_KWARGS),
    A.ShiftScaleRotate(p=1.0, scale_limit=.2, rotate_limit=0, **AUG_KWARGS),
    A.ShiftScaleRotate(p=1.0, scale_limit=0, rotate_limit=25, **AUG_KWARGS),
    A.Normalize(mean=(0.0,), std=(1,)),  # , max_pixel_value=255),
    ToTensorV2()
])#, n_aug=2)

In [6]:
proj = 'SimSiam-Lightning'

## Lightning datamodule. Comes with its own train / val / test dataloader.
mnist = MNISTDataModule('../data/', batch_size=512, train_transforms=train_transforms)
mnist.prepare_data()
mnist.setup()

## Backbone arch
cnn = CNN(C=mnist.dims[0], num_classes=mnist.num_classes)
simsiam = SimSiam(backbone=cnn, aug=alb_to_torch_aug(train_transforms))

model = BaseLitModel(
    datamodule=mnist, backbone=simsiam, loss_func=log_softmax, #metrics=metrics,
    lr=1e-3, flood_height=0.03
)

Logging metrics: []


In [7]:
tmp = mnist.val_dataloader(batch_size=32).dataset[0][0].numpy()
# tmp = mnist.val_dataloader(batch_size=32).dataset[0][0].numpy()
tmp.shape
# tmp2 = train_transforms(image=tmp.transpose(1, 2, 0))['image']
# print(tmp2.shape)
# # plt.imshow(tmp2[0]);

(1, 28, 28)

In [10]:
wandb_logger = WandbLogger(project=proj, job_type='train')
callbacks = [
    LearningRateMonitor(),  # log the LR
    ImagePredictionLogger(mnist.val_dataloader(batch_size=32), n_samples=32),
]

trainer = Trainer(
    max_epochs=200, #gpus=-1,  # all GPUs
    logger=wandb_logger, callbacks=callbacks,
    accumulate_grad_batches=1, gradient_clip_val=0,  # 0.5
    progress_bar_refresh_rate=20,
    fast_dev_run=True,
)

GPU available: True, used: False
TPU available: False, using: 0 TPU cores
Running in fast_dev_run mode: will run a full train, val and test loop using a single batch


In [11]:
# BUG TO FIX: albumentations transform expects a numpy array
# TypeError: can't convert cuda:0 device type tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.
# SOLUTION: datamodule gets the random augmentations
trainer.fit(model, mnist)

/home/freddie/venv/wotus/lib/python3.6/site-packages/pytorch_lightning/utilities/distributed.py:45: UserWarning: attribute 'datamodule' removed from hparams because it cannot be pickled
  warnings.warn(*args, **kwargs)
/home/freddie/venv/wotus/lib/python3.6/site-packages/pytorch_lightning/utilities/distributed.py:45: UserWarning: attribute 'backbone' removed from hparams because it cannot be pickled
  warnings.warn(*args, **kwargs)
wandb: wandb version 0.10.13 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade



  | Name     | Type    | Params
-------------------------------------
0 | backbone | SimSiam | 561 K 


error: OpenCV(4.4.0) /tmp/pip-req-build-hw4jq8lf/opencv/modules/imgproc/src/imgwarp.cpp:2594: error: (-215:Assertion failed) src.cols > 0 && src.rows > 0 in function 'warpAffine'


In [ ]:
%debug